   #                           INTRADAY DATA COLLECTION & ANALYSIS
   
   
Intraday data refers to data that can be collected at a very minute level. For my case, I shall be extracting data related to my heart rate, calorie burnt and step count for every minute of the day. The aim of this analysis is to:
   
   * Understand how my heart rate changes throughout the day.
   * What drives my calorie burn levels throughout the day? 
   * Can I see a particular pattern for any specific day of the week?
   * Understand how other factors impact my heart rate.
   * Understand heart rate during sleep.
   * Gain insights on how I can improve my overall heart condition.

The first part of this project involved collecting the data on a much higher level (for everyday). To view the data collection process followed by the data analysis part, refer to these links:

**[Fitbit API Data Collection](https://github.com/Arnab-Rajkhowa/Fitbit-Data-Analysis/blob/master/Fitbit%20API%20Data%20Collection.ipynb)**

**[Fitbit Exploratory Data Analysis](https://github.com/Arnab-Rajkhowa/Fitbit-Data-Analysis/blob/master/Fitbit%20Exploratory%20Data%20Analysis.ipynb)**
    

In [2]:
#import necessary libraries
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime
from datetime import timedelta
from functools import reduce


CLIENT_ID='<your client_id>'
CLIENT_SECRET='your client_secret'


server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

#Need this separately because version 1 doesn't give sleep breakdown
auth2_client_new = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')
auth2_client_new.API_VERSION = 1.2

[05/Apr/2020:18:59:26] ENGINE Listening for SIGTERM.
[05/Apr/2020:18:59:26] ENGINE Bus STARTING
[05/Apr/2020:18:59:26] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[05/Apr/2020:18:59:26] ENGINE Started monitor thread 'Autoreloader'.
[05/Apr/2020:18:59:26] ENGINE Serving on http://127.0.0.1:8080
[05/Apr/2020:18:59:26] ENGINE Bus STARTED


127.0.0.1 - - [05/Apr/2020:18:59:41] "GET /?code=c637a26ce8dcc55c879a76c07a278ecb604a870d&state=QzpvgF6NmUn8anT9FmWu2SflUXppWM HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36 Edg/80.0.361.109"


[05/Apr/2020:18:59:42] ENGINE Bus STOPPING
[05/Apr/2020:18:59:42] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[05/Apr/2020:18:59:42] ENGINE Removed handler for console events.
[05/Apr/2020:18:59:42] ENGINE Stopped thread 'Autoreloader'.
[05/Apr/2020:18:59:42] ENGINE Bus STOPPED
[05/Apr/2020:18:59:42] ENGINE Bus EXITING
[05/Apr/2020:18:59:42] ENGINE Bus EXITED
[05/Apr/2020:18:59:42] ENGINE Waiting for child threads to terminate...


### Resources for intraday time series:

1. activities/calories
2. activities/caloriesBMR
3. activities/steps
4. activities/distance
5. activities/floors
6. activities/elevation
7. activities/minutesSedentary
8. activities/minutesLightlyActive
9. activities/minutesFairlyActive
10. activities/minutesVeryActive
11. activities/activityCalories


###### ** Minimum detail level for activities/calories and activities/steps is 1min

The first step is to collect intraday data specific to heart, steps and calories. The intraday sleep data would be collected in the subsequent phase because the timestamps do not match for the cases.



#### Collecting intraday activity for heart, steps and calories.

In [43]:
#initiating empty lists to store required values
combinedall = []
date_list = []
df_list = []
combined = []

str_a = 'activities'
category = ['heart','steps', 'calories']

startTime = (datetime.date.today() - datetime.timedelta(1*365/12)).strftime("%Y-%m-%d") #date 1 month before
endTime = (datetime.date.today() - datetime.timedelta(1)).strftime("%Y-%m-%d") #yesterday's date 
dates = pd.date_range(start=startTime,end=endTime)

for c in category:
    for date in dates:
        strA = str_a + '/'+ c
        strB = str_a + '-'+ c + '-'+'intraday'
        date = date.date().strftime("%Y-%m-%d")
        data = auth2_client_new.intraday_time_series(strA,detail_level='1min')[strB]['dataset']
        df = pd.DataFrame(data)
        date_list.append(date)
        df_list.append(df)    

    for date,df in zip(date_list,df_list):
        df.loc[:,'date'] = pd.to_datetime(date) # adding the date column to the end of the dataframe
        combined.append(df)

    x = pd.concat(combined, axis = 0)
    x.rename(columns = {'value':c}, inplace = True)

   
    x.to_csv('Intraday Data\{}-intraday-{}-to-{}.csv'.format(c,startTime,endTime), sep=',')
    data = pd.DataFrame(x)
    combinedall.append(data)
    
    # making sure to clear the lists before new information is appended or else there would be duplicated records
    date_list.clear()
    df_list.clear() 
    combined.clear()
    
# joining all 3 individual tables on the 'date' and 'time' keys.
merged_data = reduce(lambda left,right: pd.merge(left,right,on=['date','time']), combinedall)
merged_data.to_csv('Intraday Data\Merged-{}-to-{}.csv'.format(startTime,endTime), sep=',')
merged_data

,time,heart,date,steps,level,mets,calories
0,00:00:00,65,2020-03-06,0,0,13,1.57664
1,00:01:00,66,2020-03-06,0,0,13,1.57664
2,00:02:00,65,2020-03-06,0,0,13,1.57664
3,00:03:00,65,2020-03-06,0,0,13,1.57664
4,00:04:00,66,2020-03-06,0,0,13,1.57664
...,...,...,...,...,...,...,...
23005,12:42:00,63,2020-04-04,14,1,28,3.39584
23006,12:43:00,63,2020-04-04,10,1,26,3.15328
23007,12:44:00,64,2020-04-04,0,0,11,1.33408
23008,12:45:00,65,2020-04-04,14,1,26,3.15328


#### The next step is to collect the sleep phase data.

This data is being collected separately because the timestamps from the sleep data do not match the time series timestamps. It would still be interesting to be able to join both the datasets to answer questions like:

 * What does my heart rate look like when sleeping?
 * How does my heart rate change with each sleep cycle?

In [4]:
sleep= []
startTime = (datetime.date.today() - datetime.timedelta(5)).strftime("%Y-%m-%d")
endTime = datetime.date.today().strftime("%Y-%m-%d") #today's date 

dates = pd.date_range(start=startTime,end=endTime)

for date in dates:
    data = auth2_client_new.sleep(date)['sleep'][0]['levels']['data']
    df = pd.DataFrame(data)
    sleep.append(df)
    
x = pd.concat(sleep, axis = 0)
x

,dateTime,level,seconds
0,2020-03-31T00:59:30.000,wake,30
1,2020-03-31T01:00:00.000,light,1680
2,2020-03-31T01:28:00.000,deep,420
3,2020-03-31T01:35:00.000,light,1740
4,2020-03-31T02:04:00.000,wake,300
...,...,...,...
23,2020-04-05T09:54:00.000,wake,420
24,2020-04-05T10:01:00.000,light,1080
25,2020-04-05T10:19:00.000,wake,420
26,2020-04-05T10:26:00.000,light,840
